In [1]:
import numpy as np
import pandas as pd

import mesa

## Agents

In [2]:
class GrassPatch(mesa.Agent):
    """
    A patch of grass that grows at a fixed rate and is eaten by sheep.
    """
    
    def __init__(self, model, fully_grown, countdown):
        """
        Creates a new patch of grass

        Args:
            fully_grown: (boolean) Whether the patch can be eaten
            countdown: (int) time for the patch of grass to be fully_grown again
        """
        super().__init__(model)
        self.fully_grown = fully_grown
        self.countdown = countdown

    def step(self):
        if not self.fully_grown:
            if self.countdown <= 0:
                self.fully_grown = True
                self.countdown = self.model.grass_regrowth_time
            else:
                self.countdown -= 1

In [5]:
#
class Sheep(mesa.Agent):

    def __init__(self, model, moore, energy=None):
        super().__init__(model)
        self.moore = moore
        self.energy = energy

    def step(self):
        """
        A Model step. Move, then eat grass and reproduce.
        """

        self.random_move()
        living = True

        # if scenario which requires sheep to eat grass for energy
        if self.model.grass:
            
            # reduce energy
            self.energy -= 1

            # if there is grass available to eat, eat it
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            grass_patch = next(
                obj for obj in this_cell if isinstance(obj, GrassPatch)
            )
            if grass_patch.fully_grown:
                # grass is eaten
                grass_patch.fully_grown = False
                
                # sheep gains energy
                self.energy += self.model.sheep_gain_from_food

        # death
        if self.energy <= 0:
            # remove agent from grid
            self.model.grid.remove_agent(self)
            
            # remove agent from model
            self.remove()

            # set `living` to false so that the reproduction does not occur
            living = False

        # reproduction chance
        if living and self.random.random() < self.model.sheep_reproduce:
            # Create a new sheep (split energy evenly between parent and child)
            if self.model.grass:
                self.energy /= 2
            lamb = Sheep(self.model, self.moore, self.energy)
            self.model.grid.place_agent(lamb, self.pos)



    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(
            self.pos, self.moore, True
        )
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

In [6]:
#
class Wolf(mesa.Agent):

    def __init__(self, model, moore, energy=None):
        super().__init__(model)
        self.moore = moore
        self.energy = energy

    def step(self):
        """
        A Model step. Move, then eat sheep and reproduce.
        """

        self.random_move()
        living = True

        # if scenario which requires sheep to eat grass for energy
        if self.model.grass:
            
            # reduce energy
            self.energy -= 1

            # if there is sheep available to eat, eat it
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            sheeps = [obj for obj in this_cell if isinstance(obj, Sheep)]
            if len(sheeps) > 0:
                # randomly choose a sheep to eat
                sheep = self.random.choice(sheeps)
                self.model.grid.remove_agent(sheep)
                sheep.remove()
                    
                # wolf gains energy
                self.energy += self.model.wolf_gain_from_food

        # death
        if self.energy <= 0:
            # remove agent from grid
            self.model.grid.remove_agent(self)
            
            # remove agent from model
            self.remove()

            # set `living` to false so that the reproduction does not occur
            living = False

        # reproduction chance
        if living and self.random.random() < self.model.wolf_reproduce:
            # Create a new sheep (split energy evenly between parent and child)
            if self.model.grass:
                self.energy /= 2
            pup = Wolf(self.model, self.moore, self.energy)
            self.model.grid.place_agent(pup, self.pos)



    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(
            self.pos, self.moore, True
        )
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

## Model

In [7]:
#
class WolfSheep(mesa.Model):
    ...